In [4]:
import os
import logging 

In [19]:
#设置日志记录 LLM --prompt question + response 记录
def setup_logger():
    """Setup logger to always print time and level."""
    logging.basicConfig(
        format='%(asctime)s %(levelname)-8s %(message)s',
        level=logging.INFO,
        datefmt='%Y-%m-%d %H:%M:%S')
    logging.getLogger().setLevel(logging.INFO)  # logging.DEBUG

#indent区分
def log_w_indent(text, indent, flag ,symbol='>>'):
    """Log and add indent."""
    ind2col = {i: f"\x1b[{a}" for i, a in enumerate([
        '1m', '31m', '33m', '34m', '35m'])}
    reset = "\x1b[0m"

    if indent > 0:
        if flag == 'Question':
            logging.info('  Q:'+
                ind2col[indent] + (indent * 2) * symbol + ' ' + text + reset)
        elif flag =='Response':
            logging.info('  LLM-A:'+
                ind2col[indent] + (indent * 2) * symbol + ' ' + text + reset)

    else:
        logging.info(ind2col[indent] + text + reset)

setup_logger()
log_w_indent('This is a test message',2,'Question')
log_w_indent('Another message without indent', 0,'Response')

INFO:root:  Q:>>>>>>>> This is a test message
INFO:root:Another message without indent


In [37]:
#配置openai客户端
from openai import OpenAI
from dotenv import load_dotenv
import os
import json
#config.json 进行配置用户openai账户 
with open(r'neuron_seg_human\llm_neuron_seg\utils\llm\llm_config.json') as config_file:
    config = json.load(config_file)

CLIENT = OpenAI(base_url=config['OPENAI_API_BASE'],
                api_key=config['OPENAI_API_KEY'])

In [41]:
# def get_sentences(response):
#     """Extract scripts & descriptions from LLM-response."""
#     # Some manual exceptions for sentence extraction.

#     # facts = response.replace('Ph.D.', 'PhD').replace('\n', ' ').split('. ')
#     # facts = [f.strip() + '.' if i < len(facts) - 1 else f.strip() for i, f in enumerate(facts)]
#     for i in facts:
#         print(i)

# def get_sentences(response):
#     """Extract scripts & descriptions from LLM-response."""
#     # Some manual exceptions for sentence extraction.
    
#     # facts = response.replace('Ph.D.', 'PhD').replace('\n', ' ').split('. ')
#     # facts = [f.strip() + '.' if i < len(facts) - 1 else f.strip() for i, f in enumerate(facts)]
#     for i in facts:
#         print(i)

from tenacity import (retry, wait_random_exponential)
@retry(wait=wait_random_exponential(min=1, max=10))
def chat_llm(prompt):
    """Predict with GPT-4 model."""

    if isinstance(prompt, str):
        messages = [
            {"role": "user", "content": prompt},
        ]
    else:
        messages = prompt

    output = CLIENT.chat.completions.create(
        model='gpt-4o',
        messages=messages,
        max_tokens=200,
    )
    response = output.choices[0].message.content
    return response

def llm_w_log(prompt, indent):
    """ log inputs and outputs."""
    log_w_indent(f'Input: {prompt}', indent , flag='Question')
    response = chat_llm(prompt)
    log_w_indent(f'Output: {response}', indent, symbol='xx',flag='Response')
    return response
llm_w_log('how to trace and restruct human neuron structure based on 3d tif images?',2)

INFO:root:  Q:>>>>>>>> Input: how to trace and restruct human neuron structure based on 3d tif images?
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:  LLM-A:xxxxxxxx Output: Tracing and reconstructing human neuron structures from 3D TIFF images is a complex but well-established process in computational neuroscience. This process involves several steps, including image preprocessing, segmentation, tracing, and visualization. Here is a general workflow to help you get started:

### 1. Image Preprocessing
The initial step involves preparing the 3D TIFF images for analysis:
- **Denoising:** Apply noise reduction techniques to improve signal-to-noise ratio.
- **Contrast Enhancement:** Adjust contrast to enhance the visibility of neuron structures.
- **Normalization:** Normalize the intensity values to a common scale.

### 2. Segmentation
Segmentation involves separating the neurons from the background.
- **Thresholding:** Use thresholdi

"Tracing and reconstructing human neuron structures from 3D TIFF images is a complex but well-established process in computational neuroscience. This process involves several steps, including image preprocessing, segmentation, tracing, and visualization. Here is a general workflow to help you get started:\n\n### 1. Image Preprocessing\nThe initial step involves preparing the 3D TIFF images for analysis:\n- **Denoising:** Apply noise reduction techniques to improve signal-to-noise ratio.\n- **Contrast Enhancement:** Adjust contrast to enhance the visibility of neuron structures.\n- **Normalization:** Normalize the intensity values to a common scale.\n\n### 2. Segmentation\nSegmentation involves separating the neurons from the background.\n- **Thresholding:** Use thresholding techniques to differentiate neurons from the background. Automatic methods such as Otsu's thresholding or adaptive thresholding can be useful.\n- **Machine Learning:** Advanced segmentation can involve machine learn